In [ ]:
import jax
import jax.numpy as jnpzzz
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import arviz as az
from jax.scipy.special import expit

import os
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

from jax import random, vmap
import jax.numpy as jnp
from jax.scipy.special import logsumexp
import plotly.express as px
import numpyro
from numpyro import handlers
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import ast
from datetime import datetime
import utils
from scipy.stats import gaussian_kde


from scipy.stats import gaussian_kde

In [3]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score,f1_score

In [4]:
import pdf as PDF
import plotly.graph_objects as go

### Conversie

In [ ]:
df3 = df2[["Studielink_Nummer","Year","has_DEFT","also_looking_elsewhere","amount_of_applied_multiple_FEB_programs",
           "StudentType","Herkomst","Opleiding",
            'preDSAC_feb', 'preDSAC_mar', 'preDSAC_apr', 'preDSAC_may', 'preDSAC_jun','preDSAC_jul']]


df3["amount_of_applied_multiple_FEB_programs"] = np.log(df3["amount_of_applied_multiple_FEB_programs"])

In [9]:
df_clusters = pd.read_excel("clusters_masters.xlsx")
name_to_id = dict(zip(df_clusters["name_program"],df_clusters["id"]))
name_to_index = dict(zip(df_clusters["name_program"],df_clusters.index))

In [ ]:
df3["studenttype_id"] = df2["StudentType"].map(name_to_id)
df3["cluster_id"] = df2["StudentType"].map(name_to_index)
df3_2024 = df3[df3["Year"] ==2024]
df3_2025 = df3[df3["Year"] ==2025]


# Individuele model

In [11]:
def model_individual(
    admitted,
    has_multiple_FEB_applications,
    has_other_NL_institution_application,
    cluster_idx,
    is_known_by_FEB,
    is_other_NL_institution,
    EER_nieuw_in_hoger_onderwijs, 
    NL_nieuw_in_hoger_onderwijs,
    rest_nieuw_in_hoger_onderwijs,
    MQP,
    hbo_instroom, 
    nietuva_student,
    uva_bachelor_student, 
    uva_master_student,
    sigma_cluster
):

    # Priors for individual-level effects
    beta_multiple_FEB = numpyro.sample("beta_multiple_FEB", dist.Normal(0, 1.5))
    beta_other_NL = numpyro.sample("beta_other_NL", dist.Normal(0, 1.5))
    
    # Priors for cluster-level effects
    gamma_intercept = numpyro.sample("gamma_intercept", dist.Normal(0, 1.5))
    gamma_known_by_FEB = numpyro.sample("gamma_known_by_FEB", dist.Normal(0, 1.5))
    gamma_other_NL_cluster = numpyro.sample("gamma_other_NL_cluster", dist.Normal(0, 1.5))

    gamma_EER_nieuw_in_hoger_onderwijs = numpyro.sample("gamma_EER_nieuw_in_hoger_onderwijs", dist.Normal(0, 1.5))
    gamma_NL_nieuw_in_hoger_onderwijs = numpyro.sample("gamma_NL_nieuw_in_hoger_onderwijs", dist.Normal(0, 1.5))
    gamma_rest_nieuw_in_hoger_onderwijs = numpyro.sample("gamma_rest_nieuw_in_hoger_onderwijs", dist.Normal(0, 1.5))
    gamma_MQP = numpyro.sample("gamma_MQP", dist.Normal(0, 1.5))
    gamma_hbo_instroom = numpyro.sample("gamma_hbo_instroom", dist.Normal(0, 1.5))
    gamma_nietuva_student = numpyro.sample("gamma_nietuva_student", dist.Normal(0, 1.5))
    gamma_uva_bachelor_student = numpyro.sample("gamma_uva_bachelor_student", dist.Normal(0, 1.5))
    gamma_uva_master_student = numpyro.sample("gamma_uva_master_student", dist.Normal(0, 1.5))

    # sigma_cluster = numpyro.sample("sigma_cluster", dist.Exponential(1))

    # Compute cluster-level mean intercepts
    cluster_intercepts_mean = (
        gamma_intercept
        + gamma_known_by_FEB * is_known_by_FEB
        + gamma_other_NL_cluster * is_other_NL_institution
        + gamma_EER_nieuw_in_hoger_onderwijs * EER_nieuw_in_hoger_onderwijs
        + gamma_NL_nieuw_in_hoger_onderwijs * NL_nieuw_in_hoger_onderwijs
        + gamma_rest_nieuw_in_hoger_onderwijs * rest_nieuw_in_hoger_onderwijs
        + gamma_MQP * MQP
        + gamma_hbo_instroom * hbo_instroom   
        + gamma_nietuva_student * nietuva_student    
        + gamma_uva_bachelor_student * uva_bachelor_student    
        + gamma_uva_master_student * uva_master_student     
    )

    cluster_intercepts = numpyro.sample(
        "cluster_intercepts",
        dist.Normal(cluster_intercepts_mean, sigma_cluster)
    )  # shape = (n_clusters,)

    # Linear predictor per student
    eta = (
        cluster_intercepts[cluster_idx]
        + beta_multiple_FEB * has_multiple_FEB_applications
        + beta_other_NL * has_other_NL_institution_application

    )
    # Likelihood
    numpyro.deterministic("logit_p", eta)

    # Likelihood
    numpyro.sample("admitted", dist.Bernoulli(logits=eta), obs=admitted)


def predict_posterior_probs_individual(
    posterior_samples,
    has_multiple_FEB_applications,
    also_looking_elsewhere,
    cluster_idx,
):
    # Expand features to match number of draws
    prob_preds = []
    # Get d-th sample of parameters
    beta_multiple = posterior_samples["beta_multiple_FEB"].mean()
    beta_other = posterior_samples["beta_other_NL"].mean()

    # Linear predictor per student
    eta = (
        posterior_samples["cluster_intercepts"].mean(axis=0)[cluster_idx]
        + beta_multiple * has_multiple_FEB_applications
        + beta_other * also_looking_elsewhere
                
    )
    prob_preds.append(expit(eta))

    # Convert to array: shape (num_draws, num_students)
    prob_preds = np.array(prob_preds)

    # Summary: mean predictive probability per student
    mean_probs = prob_preds.mean(axis=0)
    return mean_probs, prob_preds  # Optionally return full posterior predictive distribution





In [13]:

def model_levelone(tank, N, S,sigma_cluster):
    a_bar = numpyro.sample("a_bar",dist.Normal(0,1.5))
    #sigma = numpyro.sample("sigma", dist.Exponential(1))
    a = numpyro.sample("a",dist.Normal(a_bar,sigma_cluster),sample_shape=tank.shape)
    logit_p = a[tank]
    numpyro.sample("S", dist.Binomial(N,logits=logit_p), obs=S)

    p_survided = expit(logit_p)
    numpyro.deterministic("p_survided",p_survided)

def model_leveltwo(tank, N, S,is_known_by_FEB,is_other_NL_institution,sigma_cluster):
    # a_bar = numpyro.sample("a_bar",dist.Normal(0,1.5))
    # sigma = numpyro.sample("sigma", dist.Exponential(1))
    #a = numpyro.sample("a",dist.Normal(a_bar,sigma),sample_shape=tank.shape)

    # Priors for cluster-level effects
    gamma_intercept = numpyro.sample("gamma_intercept", dist.Normal(0, 1.5))
    gamma_known_by_FEB = numpyro.sample("gamma_known_by_FEB", dist.Normal(0, 1.5))
    gamma_other_NL_cluster = numpyro.sample("gamma_other_NL_cluster", dist.Normal(0, 1.5))
    # sigma_cluster = numpyro.sample("sigma_cluster", dist.Normal(0,1.5))

    cluster_intercepts_mean = (
        gamma_intercept
        + gamma_known_by_FEB * is_known_by_FEB
        + gamma_other_NL_cluster * is_other_NL_institution
    )
    
    cluster_intercepts = numpyro.sample(
        "a",
        dist.Normal(cluster_intercepts_mean, sigma_cluster)
    )  # shape = (n_clusters,)


    logit_p = cluster_intercepts[tank]
    numpyro.sample("S", dist.Binomial(N,logits=logit_p), obs=S)
    p_survided = expit(logit_p)
    numpyro.deterministic("p_survided",p_survided)


def model_levelthree(tank, N, S,is_known_by_FEB,is_other_NL_institution,
                 EER_nieuw_in_hoger_onderwijs, 
                 NL_nieuw_in_hoger_onderwijs,
                 rest_nieuw_in_hoger_onderwijs,
                 MQP, hbo_instroom, herinschrijver,
                nietuva_student,
                 uva_bachelor_student, uva_master_student,
                 sigma_cluster):
    # a_bar = numpyro.sample("a_bar",dist.Normal(0,1.5))
    # sigma = numpyro.sample("sigma", dist.Exponential(1))
    #a = numpyro.sample("a",dist.Normal(a_bar,sigma),sample_shape=tank.shape)

    # Priors for cluster-level effects
    gamma_intercept = numpyro.sample("gamma_intercept", dist.Normal(0, 1.5))
    gamma_known_by_FEB = numpyro.sample("gamma_known_by_FEB", dist.Normal(0, 1.5))
    gamma_other_NL_cluster = numpyro.sample("gamma_other_NL_cluster", dist.Normal(0, 1.5))

    gamma_EER_nieuw_in_hoger_onderwijs = numpyro.sample("gamma_EER_nieuw_in_hoger_onderwijs", dist.Normal(0, 1.5))
    gamma_NL_nieuw_in_hoger_onderwijs = numpyro.sample("gamma_NL_nieuw_in_hoger_onderwijs", dist.Normal(0, 1.5))
    gamma_rest_nieuw_in_hoger_onderwijs = numpyro.sample("gamma_rest_nieuw_in_hoger_onderwijs", dist.Normal(0, 1.5))
    gamma_MQP = numpyro.sample("gamma_MQP", dist.Normal(0, 1.5))
    gamma_hbo_instroom = numpyro.sample("gamma_hbo_instroom", dist.Normal(0, 1.5))
    gamma_herinschrijver = numpyro.sample("gamma_herinschrijver", dist.Normal(0, 1.5))
    gamma_nietuva_student = numpyro.sample("gamma_nietuva_student", dist.Normal(0, 1.5))
    gamma_uva_bachelor_student = numpyro.sample("gamma_uva_bachelor_student", dist.Normal(0, 1.5))
    gamma_uva_master_student = numpyro.sample("gamma_uva_master_student", dist.Normal(0, 1.5))

    # sigma_cluster = numpyro.sample("sigma_cluster", dist.Normal(0,1.5))

    cluster_intercepts_mean = (
        gamma_intercept
        + gamma_known_by_FEB * is_known_by_FEB
        + gamma_other_NL_cluster * is_other_NL_institution
        + gamma_EER_nieuw_in_hoger_onderwijs * EER_nieuw_in_hoger_onderwijs
        + gamma_NL_nieuw_in_hoger_onderwijs * NL_nieuw_in_hoger_onderwijs
        + gamma_rest_nieuw_in_hoger_onderwijs * rest_nieuw_in_hoger_onderwijs
        + gamma_MQP * MQP
        + gamma_hbo_instroom * hbo_instroom
        + gamma_herinschrijver * herinschrijver     
        + gamma_nietuva_student * nietuva_student    
        + gamma_uva_bachelor_student * uva_bachelor_student    
        + gamma_uva_master_student * uva_master_student     
    )
    
    cluster_intercepts = numpyro.sample(
        "a",
        dist.Normal(cluster_intercepts_mean, sigma_cluster)
    )  # shape = (n_clusters,)

    logit_p = cluster_intercepts[tank]
    numpyro.sample("S", dist.Binomial(N,logits=logit_p), obs=S)
    p_survided = expit(logit_p)
    numpyro.deterministic("p_survided",p_survided)




def predict_posterior_probs_levelone(
    posterior_samples,
    has_multiple_FEB_applications,
    also_looking_elsewhere,
    cluster_idx,
):
    # Expand features to match number of draws
    prob_preds = []
    # Get d-th sample of parameters
    # beta_multiple = posterior_samples["beta_multiple_FEB"].mean()
    # beta_other = posterior_samples["beta_other_NL"].mean()

    # Linear predictor per student
    eta = (
        posterior_samples["a"].mean(axis=0)[cluster_idx]
        # + beta_multiple * has_multiple_FEB_applications
        # + beta_other * also_looking_elsewhere
                
    )
    prob_preds.append(expit(eta))

    # Convert to array: shape (num_draws, num_students)
    prob_preds = np.array(prob_preds)

    # Summary: mean predictive probability per student
    mean_probs = prob_preds.mean(axis=0)
    return mean_probs, prob_preds  # Optionally return full posterior predictive distribution





In [14]:
def runmodel_individual(df3_2024,sigma = 0.1,label="jan"):
  dat = dict(admitted=df3_2024["has_DEFT"].astype(int).values,
            has_multiple_FEB_applications=df3_2024["amount_of_applied_multiple_FEB_programs"].values,
            has_other_NL_institution_application = df3_2024["also_looking_elsewhere"].values,
              cluster_idx= df3_2024["cluster_id"].values,
                is_known_by_FEB= df_clusters["is_known_by_FEB"].values,
                is_other_NL_institution= df_clusters["is_other_NL_institution"].values,
                EER_nieuw_in_hoger_onderwijs =df_clusters["EER_nieuw_in_hoger_onderwijs"].values,
                NL_nieuw_in_hoger_onderwijs =df_clusters["NL_nieuw_in_hoger_onderwijs"].values, 
                rest_nieuw_in_hoger_onderwijs =df_clusters["rest_nieuw_in_hoger_onderwijs"].values,
                MQP =df_clusters["MQP"].values, 
                hbo_instroom =df_clusters["hbo_instroom"].values,    
                nietuva_student =df_clusters["nietuva_student"].values,  
                uva_bachelor_student =df_clusters["uva_bachelor_student"].values, 
                uva_master_student =df_clusters["uva_master_student"].values,
                sigma_cluster = sigma                                                                    
                    )


  mcmc = MCMC(NUTS(model_individual), num_warmup=100,num_samples=200,num_chains=4,progress_bar=False)
  mcmc.run(random.PRNGKey(0), **dat)
  # model_multilevel.print_summary()
  posterior_samples = mcmc.get_samples()


  mean_probs,prob_preds = predict_posterior_probs_individual(
      posterior_samples,
      df3_2024["amount_of_applied_multiple_FEB_programs"].values,
      df3_2024["also_looking_elsewhere"].values,
      df3_2024["cluster_id"].values,
  )
  df3_2024[f"model_{label}"] = mean_probs


  return df3_2024, posterior_samples


def runmodel_levelone(df3_2024,sigma = 0.1,label="jan"):

  df4 = pd.DataFrame()
  df4["StudentType"] = df_clusters[["name_program"]]
  df4["sum"] = 0
  df4["count"] = 0
  df4["Conversie"] = 0
  df4["cluster_id"] = range(0,len(df4))
  temp = df3_2024.groupby(["StudentType"])["has_DEFT"].agg(["sum","count"]).reset_index()
  temp["Conversie"] = temp["sum"]/temp["count"]
  temp["cluster_id"] = temp["StudentType"].map(name_to_id)
  df4 = df4.merge(temp,on="StudentType", how="left",suffixes=("","_new"))

  df4["sum"] = df4["sum_new"].combine_first(df4["sum"])
  df4["count"] = df4["count_new"].combine_first(df4["count"])
  df4["Conversie"] = df4["Conversie_new"].combine_first(df4["Conversie"])
  df4 = df4[["StudentType","sum","count","Conversie","cluster_id"]]



  dat = dict(S=df4["sum"].values,N=df4["count"].values, tank= df4["cluster_id"].values, sigma_cluster=sigma)


  mcmc = MCMC(NUTS(model_levelone), num_warmup=500,num_samples=500,num_chains=4,progress_bar=False)
  mcmc.run(random.PRNGKey(0), **dat)
  # model_multilevel.print_summary()
  posterior_samples = mcmc.get_samples()

  df4[f"model_{label}"] = expit(jnp.mean(posterior_samples["a"], 0))


  return df4, posterior_samples

def runmodel_leveltwo(df3_2024,sigma = 0.1,label="jan"):

  df4 = pd.DataFrame()
  df4["StudentType"] = df_clusters[["name_program"]]
  df4["sum"] = 0
  df4["count"] = 0
  df4["Conversie"] = 0
  df4["cluster_id"] = range(0,len(df4))
  temp = df3_2024.groupby(["StudentType"])["has_DEFT"].agg(["sum","count"]).reset_index()
  temp["Conversie"] = temp["sum"]/temp["count"]
  temp["cluster_id"] = temp["StudentType"].map(name_to_id)
  df4 = df4.merge(temp,on="StudentType", how="left",suffixes=("","_new"))

  df4["sum"] = df4["sum_new"].combine_first(df4["sum"])
  df4["count"] = df4["count_new"].combine_first(df4["count"])
  df4["Conversie"] = df4["Conversie_new"].combine_first(df4["Conversie"])
  df4 = df4[["StudentType","sum","count","Conversie","cluster_id"]]



  dat = dict(S=df4["sum"].values,N=df4["count"].values, tank= df4["cluster_id"].values,
             is_known_by_FEB= df_clusters["is_known_by_FEB"].values,
               is_other_NL_institution =df_clusters["is_other_NL_institution"].values,
                 sigma_cluster=sigma)


  mcmc = MCMC(NUTS(model_leveltwo), num_warmup=500,num_samples=500,num_chains=4,progress_bar=False)
  mcmc.run(random.PRNGKey(0), **dat)
  # model_multilevel.print_summary()
  posterior_samples = mcmc.get_samples()

  df4[f"model_{label}"] = expit(jnp.mean(posterior_samples["a"], 0))


  return df4, posterior_samples


def runmodel_levelthree(df3_2024,sigma = 0.1,label="jan"):

  df4 = pd.DataFrame()
  df4["StudentType"] = df_clusters[["name_program"]]
  df4["sum"] = 0
  df4["count"] = 0
  df4["Conversie"] = 0
  df4["cluster_id"] = range(0,len(df4))
  temp = df3_2024.groupby(["StudentType"])["has_DEFT"].agg(["sum","count"]).reset_index()
  temp["Conversie"] = temp["sum"]/temp["count"]
  temp["cluster_id"] = temp["StudentType"].map(name_to_id)
  df4 = df4.merge(temp,on="StudentType", how="left",suffixes=("","_new"))

  df4["sum"] = df4["sum_new"].combine_first(df4["sum"])
  df4["count"] = df4["count_new"].combine_first(df4["count"])
  df4["Conversie"] = df4["Conversie_new"].combine_first(df4["Conversie"])
  df4 = df4[["StudentType","sum","count","Conversie","cluster_id"]]



  dat = dict(S=df4["sum"].values,N=df4["count"].values, tank= df4["cluster_id"].values,
             is_known_by_FEB= df_clusters["is_known_by_FEB"].values,
               is_other_NL_institution =df_clusters["is_other_NL_institution"].values,
               EER_nieuw_in_hoger_onderwijs =df_clusters["EER_nieuw_in_hoger_onderwijs"].values,
               NL_nieuw_in_hoger_onderwijs =df_clusters["NL_nieuw_in_hoger_onderwijs"].values, 
               rest_nieuw_in_hoger_onderwijs =df_clusters["rest_nieuw_in_hoger_onderwijs"].values,
               MQP =df_clusters["MQP"].values, 
               hbo_instroom =df_clusters["hbo_instroom"].values,    
               herinschrijver =df_clusters["herinschrijver"].values, 
               nietuva_student =df_clusters["nietuva_student"].values,  
               uva_bachelor_student =df_clusters["uva_bachelor_student"].values, 
               uva_master_student =df_clusters["uva_master_student"].values,

                 sigma_cluster=sigma)


  mcmc = MCMC(NUTS(model_levelthree), num_warmup=500,num_samples=500,num_chains=4,progress_bar=False)
  mcmc.run(random.PRNGKey(0), **dat)
  # model_multilevel.print_summary()
  posterior_samples = mcmc.get_samples()

  df4[f"model_{label}"] = expit(jnp.mean(posterior_samples["a"], 0))


  return df4, posterior_samples






In [15]:
df3_2024_jan = df3_2024.copy()
df3_2024_feb = df3_2024[df3_2024["preDSAC_feb"] ==False]
df3_2024_mar = df3_2024[df3_2024["preDSAC_feb"] ==False]
df3_2024_apr = df3_2024[df3_2024["preDSAC_feb"] ==False]
df3_2024_may = df3_2024[df3_2024["preDSAC_may"] ==False]
df3_2024_jun = df3_2024[df3_2024["preDSAC_jun"] ==False]
df3_2024_jul = df3_2024[df3_2024["preDSAC_jul"] ==False]

df3_2025_jan = df3_2025.copy()
df3_2025_feb = df3_2025[df3_2025["preDSAC_feb"] ==False]
df3_2025_mar = df3_2025[df3_2025["preDSAC_feb"] ==False]
df3_2025_apr = df3_2025[df3_2025["preDSAC_feb"] ==False]
df3_2025_may = df3_2025[df3_2025["preDSAC_may"] ==False]
df3_2025_jun = df3_2025[df3_2025["preDSAC_jun"] ==False]
df3_2025_jul = df3_2025[df3_2025["preDSAC_jul"] ==False]
# df3_2024_jan,posterior_samples_jan, idata_jan = runmodel(df3_2024_jan, "jan")
# df3_2024_jul,posterior_samples_jul,idata_jul = runmodel(df3_2024_jun,"jul")


In [16]:
# az.plot_trace(idata_jul,compact=True, var_names = ["cluster_intercepts"])

In [17]:
def predict_individual(df3_2025, posterior_samples,label="jan"):
    mean_probs,prob_preds = predict_posterior_probs_individual(
    posterior_samples,
    df3_2025["amount_of_applied_multiple_FEB_programs"].values,
    df3_2025["also_looking_elsewhere"].values,
    df3_2025["cluster_id"].values,)
    df3_2025[f"model_{label}"] = mean_probs

    return df3_2025

def predict_levelone(df3_2025, posterior_samples,label="jan"):

    
    mean_probs,prob_preds = predict_posterior_probs_levelone(
    posterior_samples,
    df3_2025["amount_of_applied_multiple_FEB_programs"].values,
    df3_2025["also_looking_elsewhere"].values,
    df3_2025["cluster_id"].values,)

    df3_2025[f"model_{label}"] = mean_probs

    return df3_2025


In [18]:
train = df3_2024_jul.copy()

In [19]:
test = df3_2025_jul.copy()
test = test.drop(columns=['preDSAC_feb', 'preDSAC_mar', 'preDSAC_apr', 'preDSAC_may',
       'preDSAC_jun', 'preDSAC_jul'])

In [ ]:
train_basic_1,_ = runmodel_levelone(df3_2024_jul.copy(),0.1, "basic_1")
test = predict_levelone(test,_, "basic_1")

train_basic_5,_ = runmodel_levelone(df3_2024_jul.copy(),0.5, "basic_5")
test = predict_levelone(test,_, "basic_5")


train_basic_10,_ = runmodel_levelone(df3_2024_jul.copy(),1, "basic_10")
test = predict_levelone(test,_, "basic_10")

train_basic_15,_ = runmodel_levelone(df3_2024_jul.copy(),1.5, "basic_15")
test = predict_levelone(test,_, "basic_15")

train_type_1,_ = runmodel_levelthree(df3_2024_jul.copy(),0.1, "type_1")
test = predict_levelone(test,_, "type_1")

train_type_5,_ = runmodel_levelthree(df3_2024_jul.copy(),0.5, "type_5")
test = predict_levelone(test,_, "type_5")

train_type_10,_ = runmodel_levelthree(df3_2024_jul.copy(),1.0, "type_10")
test = predict_levelone(test,_, "type_10")

train_type_15,_ = runmodel_levelthree(df3_2024_jul.copy(),1.5, "type_15")
test = predict_levelone(test,_, "type_15")

train_individual,_ = runmodel_individual(df3_2024_jul.copy(),.1, "individual_1")
test = predict_individual(test,_, "individual_1")

train_individual,_ = runmodel_individual(train_individual,.5, "individual_5")
test = predict_individual(test,_, "individual_5")

train_individual,_ = runmodel_individual(train_individual,1.0, "individual_10")
test = predict_individual(test,_, "individual_10")

train_individual,_ = runmodel_individual(train_individual,1.5, "individual_15")
test = predict_individual(test,_, "individual_15")



In [21]:
from scipy.stats import norm

def poisson_binomial_interval(pred_probs,alpha=0.05):
    pred_probs = np.asanyarray(pred_probs)
    mean = np.sum(pred_probs)
    var = np.sum(pred_probs*(1-pred_probs))
    std = np.sqrt(var)

    z = norm.ppf(1-alpha/2)
    ci_lower = mean -z*std
    ci_upper = mean + z*std

    return {
        "mean": np.round(mean,0),
        "std":np.round(std,5),
        "ci_lower":np.round(ci_lower,0),
        "ci_upper":np.round(ci_upper,0),
        "combined":f"{int(np.round(ci_lower,0))}-{int(np.round(ci_upper,0))}"

    }

In [ ]:
Programs = ['M Business Administration', 'M Finance',
       'M Data Science & Bus Analytics', 'M Economics',
       'M Accountancy and Control', 'M Business Economics', 'M Econometrics',
       'M Actuarial Sc & Math Finance', 'M Fiscal Economics', "M Business Information Techn M"]

models = ['model_basic_1', 'model_basic_5', 'model_basic_10',
       'model_basic_15', 'model_type_1', 'model_type_5', 'model_type_10',
       'model_type_15', 'model_individual_1','model_individual_5','model_individual_10','model_individual_15']

rows = []

for model in models:

    row = {"model":model}

    for program in Programs:
        
        df_program = test[test["Opleiding"] == program]

        row[program] = poisson_binomial_interval(df_program[model])["combined"]

    rows.append(row)



In [24]:
pd.DataFrame(rows).to_excel("CI95.xlsx",index=False)